<a href="https://colab.research.google.com/github/cygkichi/dl4us_test/blob/master/smartmill_ver03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
""" Googleドライブから音声ファイルをダウンロード
"""
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/smartmill

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
dryer_sample		     R20190117204623_last02.wav
I20190118194700_First00.wav  R20190117204803_last01.wav
I20190118215800_First01.wav  R20190117204858_last00.wav
I20190118221500_last.wav


In [2]:
import numpy as np
import scipy.signal
import glob
import os
import scipy.io.wavfile
from scipy.io import wavfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import IPython.display
#IPython.display.Audio(wavfile)

import librosa
import numpy as np
import scipy
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras.utils import Sequence, to_categorical

Using TensorFlow backend.


In [0]:
class Config(object):
    def __init__(self,
                 sampling_rate=16000, audio_duration=2, n_classes=41,
                 use_mfcc=False, n_folds=10, learning_rate=0.0001, 
                 max_epochs=50, n_mfcc=20):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_mfcc = use_mfcc
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs

        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)

In [0]:
class DataGenerator(Sequence):
    def __init__(self, config, data_dir, list_IDs, labels=None, 
                 batch_size=64, preprocessing_fn=lambda x: x):
        self.config = config
        self.data_dir = data_dir
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.preprocessing_fn = preprocessing_fn
        self.on_epoch_end()
        self.dim = self.config.dim

    def __len__(self):
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        return self.__data_generation(list_IDs_temp)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        cur_batch_size = len(list_IDs_temp)
        X = np.empty((cur_batch_size, *self.dim))

        input_length = self.config.audio_length
        for i, ID in enumerate(list_IDs_temp):
            file_path = self.data_dir + ID

            # Read and Resample the audio
            data, _ = librosa.core.load(file_path, sr=self.config.sampling_rate,
                                        res_type='kaiser_fast')

            # Random offset / Padding
            if len(data) > input_length:
                max_offset = len(data) - input_length
                offset = np.random.randint(max_offset)
                data = data[offset:(input_length+offset)]
            else:
                if input_length > len(data):
                    max_offset = input_length - len(data)
                    offset = np.random.randint(max_offset)
                else:
                    offset = 0
                data = np.pad(data, (offset, input_length - len(data) - offset), "constant")

            # Normalization + Other Preprocessing
            if self.config.use_mfcc:
                data = librosa.feature.mfcc(data, sr=self.config.sampling_rate,
                                                   n_mfcc=self.config.n_mfcc)
                data = np.expand_dims(data, axis=-1)
            else:
                data = self.preprocessing_fn(data)[:, np.newaxis]
            X[i,] = data

        if self.labels is not None:
            y = np.empty(cur_batch_size, dtype=int)
            for i, ID in enumerate(list_IDs_temp):
                y[i] = self.labels[ID]
            return X, to_categorical(y, num_classes=self.config.n_classes)
        else:
            return X

In [0]:
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+1e-6)
    return data-0.5

In [0]:
def get_1d_conv_model(config):

    nclass = config.n_classes
    input_length = config.audio_length

    inp = Input(shape=(input_length,1))
    x = Convolution1D(16, 9, activation=relu, padding="valid")(inp)
    x = Convolution1D(16, 9, activation=relu, padding="valid")(x)
    x = MaxPool1D(16)(x)
    x = Dropout(rate=0.1)(x)

    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = MaxPool1D(4)(x)
    x = Dropout(rate=0.1)(x)

    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = MaxPool1D(4)(x)
    x = Dropout(rate=0.1)(x)

    x = Convolution1D(256, 3, activation=relu, padding="valid")(x)
    x = Convolution1D(256, 3, activation=relu, padding="valid")(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(rate=0.2)(x)

    x = Dense(64, activation=relu)(x)
    x = Dense(1028, activation=relu)(x)
    out = Dense(nclass, activation=softmax)(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(config.learning_rate)

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    return model

In [0]:
class WaveCollector(object):
    """ Main Preprocessing class.
    """
    def __init__(self):
        self.fs        = 44100
        self.nfile     = 0
        self.labellist = []
        self.filename  = []
        self.data      = []
        self.label     = []

    def read(self, filename, label, length=None):
        rate, data = wavfile.read(filename)
        
        """TODO : サンプリング周波数が異なるときの処理"""
        """TODO : BITが異なるときの処理"""
        data = data / 32768
        isStereo = (lambda data: len(data.shape) == 2)
        if isStereo(data):
            data = data[:,0]

        if length is not None:
            data = data[:length]
            
        self.filename.append(filename)
        self.data.append(data)
        self.label.append(label)
        self.nfile += 1
        
    def data_split(self, test_size=0.4):
        self.labellist = list(set(self.label))
        label_idx = {label:i for i, label in enumerate(self.labellist)}
        X     = np.array(self.data)
        y     = np.array([label_idx[l] for l in self.label])
        fname = np.array(self.filename)
        train_idx = np.ones(self.nfile, dtype=np.bool)
        train_idx[:int(test_size * self.nfile)] = False
        np.random.shuffle(train_idx)
        test_idx = np.invert(train_idx)

        self.X_train     = X[train_idx]
        self.y_train     = y[train_idx]
        self.fname_train = fname[train_idx]
        self.X_test      = X[test_idx]
        self.y_test      = y[test_idx]
        self.fname_test  = fname[test_idx]

In [8]:
""" RUN ML
"""


COMPLETE_RUN = True
length = None
if not COMPLETE_RUN:
    length = 100

filenames = glob.glob('/content/drive/My Drive/smartmill/dryer_sample/*.wav')
filenames.sort()

wc = WaveCollector()
print(filenames)
for f in filenames:
    label = f.split('/')[-1].split('_')[0]
    wc.read(f, label)

wc.data_split()
config = Config(sampling_rate=16000, audio_duration=2, n_folds=10, learning_rate=0.001)


['/content/drive/My Drive/smartmill/dryer_sample/cold_01_01.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_02.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_03.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_04.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_05.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_06.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_07.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_08.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_09.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_10.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_11.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_12.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_13.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_14.wav', '/content/drive/My Drive/smartmill/dryer_sample/cold_01_15.wav', '/content/drive/My Drive

In [0]:
from keras.layers import Dense
from keras.models import Sequential
def get_dense_model(nclass):
    
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=500))
    model.add(Dense(units=nclass, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
    
    return model


from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers import Dense, Activation, Flatten, Dropout, Conv1D
from keras.layers.pooling import MaxPooling1D

def get_1d_conv_model(nclass=2, audio_length=400):
    model = Sequential()
    model.add(Conv1D(filters=400, kernel_size=20))
    model.add(Activation('relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='mse',optimizer='sgd',metrics=['accuracy'])


    return model


In [0]:
x = np.array([a[:400] for a in wc.X_train])
y = np.eye(2)[wc.y_train]

a,b = x.shape
x = x.reshape(a,b,1)

In [16]:

model = get_1d_conv_model()
model.fit(x, y, epochs=500)


Epoch 1/500
35/35 [==============================] - 1s 16ms/step - loss: 0.2502 - acc: 0.5143
Epoch 2/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2498 - acc: 0.5429
Epoch 3/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2498 - acc: 0.5429
Epoch 4/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2500 - acc: 0.5429
Epoch 5/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2501 - acc: 0.4000
Epoch 6/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2497 - acc: 0.5714
Epoch 7/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2494 - acc: 0.5429
Epoch 8/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2492 - acc: 0.5429
Epoch 9/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2496 - acc: 0.5429
Epoch 10/500
35/35 [==============================] - 0s 1ms/step - loss: 0.2494 - acc: 0.8286
Epoch 11/500
35/35 [==============================] - 0s 1